# Prepair

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Warning copy ferdousi.txt in this directory or copy form google drive form correct path!


In [14]:
# !cp /content/drive/MyDrive/HW4_Q1/ferdousi.txt .
!cp /content/drive/MyDrive/HW4_Q1_notebook/ferdousi.txt .

In [3]:
!pip install datasets
!pip install sacrebleu
!pip install tensorboard


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 19.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 3.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
%load_ext tensorboard

In [5]:
from __future__ import unicode_literals, print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

import datasets
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import re
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
import gdown

In [7]:
gdown.download(url="https://github.com/Text-Mining/Persian-Wikipedia-Corpus/raw/master/models/glove/vectors.zip",output="vectors.zip")
!unzip "vectors.zip"

Downloading...
From: https://github.com/Text-Mining/Persian-Wikipedia-Corpus/raw/master/models/glove/vectors.zip
To: /content/vectors.zip
100%|██████████| 48.2M/48.2M [00:00<00:00, 203MB/s]


Archive:  vectors.zip
  inflating: vectors.txt             


# Data PreProcessing

In [15]:
SOS_token = 0
EOS_token = 1
PAD_token = 2
MAX_LENGTH = 0

#initialize Lang Class
class Lang:
   def __init__(self):
       #initialize containers to hold the words and corresponding index
       self.word2index = {}
       self.word2count = {}
       self.index2word = {0: "SOS", 1: "EOS", 2:"PAD"} #
       self.n_words = 3  # Count SOS and EOS and PAD

#split a sentence into words and add it to the container
   def addSentence(self, sentence):
       for word in sentence.split(' '):
           self.addWord(word)

#If the word is not in the container, the word will be added to it, 
#else, update the word counter
   def addWord(self, word):
       if word not in self.word2index:
           self.word2index[word] = self.n_words
           self.word2count[word] = 1
           self.index2word[self.n_words] = word
           self.n_words += 1
       else:
           self.word2count[word] += 1

In [16]:
def read_sentence(df, lang1, lang2):
   sentence1 = df[lang1]
   sentence2 = df[lang2]
   return sentence1, sentence2
   
def Word2vec(path="/content/vectors.txt"):
  file=open(path, mode="r")
  L=file.readlines(-1)
  Word_feature={}
  for i in L:
    vect=i.split()
    Word_feature[vect[0]]=np.array(vect[1:],dtype=float)
  return Word_feature

def read_file(loc, lang1, lang2):
   df = pd.read_csv(loc, delimiter='\n', header=None, names=['Beyt'])[2::]
   d = {lang1:df[0::2].values[:,0].tolist() , lang2: df[1::2].values[:,0].tolist()}
   df=pd.DataFrame(d)
   return df

def process_data(lang1,lang2):
   global MAX_LENGTH
   df = read_file('ferdousi.txt', lang1, lang2)
   print("Read %s sentence pairs" % len(df))
   sentence1, sentence2 = read_sentence(df, lang1, lang2)
  #  print(sentence1[0])
   source = Lang()
   target = Lang()
   pairs = []

   for i in range(len(df)):
      if len(sentence1[i].split(' '))>MAX_LENGTH:
         MAX_LENGTH = len(sentence1[i].split(' '))
     
      full = [sentence1[i], sentence2[i]]
      source.addSentence(sentence1[i])
      target.addSentence(sentence2[i])
      pairs.append(full)

   print(f"Maximum lenght of input is {MAX_LENGTH}")

   return source, target, pairs

In [17]:
lang1 = 'M1'
lang2 = 'M2'
# a=read_file('ferdousi.txt', lang1, lang2)
source, target, pairs = process_data(lang1, lang2)

Read 49609 sentence pairs
Maximum lenght of input is 11


In [18]:
def indexesFromSentence(lang, sentence):
   return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
   global PAD_token
   global MAX_LENGTH
   indexes = indexesFromSentence(lang, sentence)
   pad_list=[PAD_token for i in range(MAX_LENGTH-len(indexes))]
   indexes =  indexes  + pad_list
   indexes.append(EOS_token)
   return torch.tensor(indexes, dtype=torch.long,device=device).view(-1, 1)

def tensorsFromPair(input_lang, output_lang, pair):
   input_tensor = tensorFromSentence(input_lang, pair[0])
   target_tensor = tensorFromSentence(output_lang, pair[1])
   return (input_tensor, target_tensor)

In [19]:
training_pairs = [tensorsFromPair(source, target, random.choice(pairs))
                     for i in range(10)]
                     

In [20]:
training_pairs

[(tensor([[ 106],
          [1269],
          [  48],
          [ 176],
          [ 705],
          [  37],
          [1161],
          [   2],
          [   2],
          [   2],
          [   2],
          [   1]], device='cuda:0'), tensor([[ 110],
          [  83],
          [1429],
          [  13],
          [  83],
          [4805],
          [   2],
          [   2],
          [   2],
          [   2],
          [   2],
          [   1]], device='cuda:0')), (tensor([[1420],
          [   7],
          [3896],
          [   7],
          [2071],
          [ 490],
          [   2],
          [   2],
          [   2],
          [   2],
          [   2],
          [   1]], device='cuda:0'), tensor([[  52],
          [4177],
          [ 184],
          [ 183],
          [ 208],
          [ 705],
          [ 705],
          [   2],
          [   2],
          [   2],
          [   2],
          [   1]], device='cuda:0')), (tensor([[ 328],
          [  83],
          [5853],
          

# Model

In [21]:
class Encoder(nn.Module):
   def __init__(self, input_dim, hidden_dim, embbed_dim, num_layers,GRU=False):
       super(Encoder, self).__init__()
      
       #set the encoder input dimesion , embbed dimesion, hidden dimesion, and number of layers 
       self.input_dim = input_dim
       self.embbed_dim = embbed_dim
       self.hidden_dim = hidden_dim
       self.num_layers = num_layers

       #initialize the embedding layer with input and embbed dimention
       self.embedding = nn.Embedding(input_dim, self.embbed_dim,padding_idx=2) #
       #intialize the GRU to take the input dimetion of embbed, and output dimention of hidden and
       #set the number of gru layers
       if GRU:
        self.recurrent = nn.GRU(self.embbed_dim, self.hidden_dim, num_layers=self.num_layers)
       else:
        self.recurrent = nn.LSTM(self.embbed_dim, self.hidden_dim, num_layers=self.num_layers)
              
   def forward(self, src):
       
       embedded = self.embedding(src).view(1,1,-1)
       outputs, hidden = self.recurrent(embedded)
       return outputs, hidden
   def initHidden(self):
     return torch.zeros(1, 1, self.hidden_dim, device=device)

class Decoder(nn.Module):
   def __init__(self, output_dim, hidden_dim, embbed_dim, num_layers,GRU=False):
       super(Decoder, self).__init__()

#set the encoder output dimension, embed dimension, hidden dimension, and number of layers 
       self.embbed_dim = embbed_dim
       self.hidden_dim = hidden_dim
       self.output_dim = output_dim
       self.num_layers = num_layers

# initialize every layer with the appropriate dimension. For the decoder layer, it will consist of an embedding, GRU, a Linear layer and a Log softmax activation function.
       self.embedding = nn.Embedding(output_dim, self.embbed_dim ,padding_idx=2) # 
       
       if GRU:
        self.recurrent = nn.GRU(self.embbed_dim, self.hidden_dim, num_layers=self.num_layers)
       else :
        self.recurrent=nn.LSTM(self.embbed_dim, self.hidden_dim, num_layers=self.num_layers)

       self.out = nn.Linear(self.hidden_dim, output_dim)
       self.softmax = nn.LogSoftmax(dim=1)
      
   def forward(self, input, hidden):

# reshape the input to (1, batch_size)
       input = input.view(1, -1)
       embedded = F.relu(self.embedding(input))
       output, hidden = self.recurrent(embedded, hidden)       
       prediction = self.softmax(self.out(output[0]))
      
       return prediction, hidden
   def initHidden(self):
      return torch.zeros(1, 1, self.hidden_dim, device=device)

class Seq2Seq(nn.Module):
   def __init__(self, encoder, decoder, device, MAX_LENGTH=MAX_LENGTH,mode='GRU'):
       super().__init__()
      
#initialize the encoder and decoder
       self.encoder = encoder
       self.decoder = decoder
       self.device = device
       self.mode = mode
   def forward(self, source, target, teacher_forcing_ratio=0.5):

       input_length = source.size(0) #get the input length (number of words in sentence)
       batch_size = target.shape[1] 
       target_length = target.shape[0]
       vocab_size = self.decoder.output_dim
      
#initialize a variable to hold the predicted outputs
       outputs = torch.zeros(target_length, batch_size, vocab_size).to(self.device)

#encode every word in a sentence
       for i in range(input_length):
           encoder_output, encoder_hidden = self.encoder(source[i])

#use the encoder’s hidden layer as the decoder hidden
       
       if self.mode == "GRU":
         decoder_hidden = encoder_hidden.to(device)
       elif self.mode == "LSTM":
         decoder_hidden = (encoder_hidden[0].to(device),encoder_hidden[1].to(device))
       
#add a token before the first predicted word
       decoder_input = torch.tensor([SOS_token], device=device)  # SOS

#topk is used to get the top K value over a list
#predict the output word from the current target word. If we enable the teaching force,  then the #next decoder input is the next word, else, use the decoder output highest value. 

       for t in range(target_length):   
           decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
           outputs[t] = decoder_output
           teacher_force = random.random() < teacher_forcing_ratio
           topv, topi = decoder_output.topk(1)
           input = (target[t] if teacher_force else topi)
           if(teacher_force == False and input.item() == EOS_token):
               break

       return outputs

# Train

In [22]:
teacher_forcing_ratio = 0.5
import datasets

metric = datasets.load_metric('sacrebleu')

def clacModel(model, input_tensor, target_tensor, model_optimizer, criterion):

   model_optimizer.zero_grad()

   input_length = input_tensor.size(0)
   loss = 0
   epoch_loss = 0
   # print(input_tensor.shape)

   output = model(input_tensor, target_tensor,teacher_forcing_ratio)

   num_iter = output.size(0)
  #  print(num_iter)

#calculate the loss from a predicted sentence with the expected result
   for ot in range(num_iter):
       loss += criterion(output[ot], target_tensor[ot])
       
   metric.add(predictions=output, references=target_tensor)
   
   loss.backward()
   model_optimizer.step()
   epoch_loss = loss.item() / num_iter

   return epoch_loss,metric

def trainModel(model, source, target, pairs, num_iteration=20000,type='GRU'):
   
   model.train()

  #  optimizer = optim.SGD(model.parameters(), lr=0.01)
   optimizer = optim.Adam(model.parameters(), lr=0.001,weight_decay=1e-5)
  #  scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=num_iteration//5000,eta_min=1e-4,last_epoch=-1)
  #  criterion = nn.NLLLoss()
   criterion = nn.CrossEntropyLoss()
   total_loss_iterations = 0
   final_score = 0
   training_pairs = [tensorsFromPair(source, target, random.choice(pairs))
                     for i in range(num_iteration)]
  
   for iter in tqdm(range(1, num_iteration+1)):
       training_pair = training_pairs[iter - 1]
       input_tensor = training_pair[0]
       target_tensor = training_pair[1]

       loss,metric = clacModel(model, input_tensor, target_tensor, optimizer, criterion)
       s  = metric.compute()

       final_score += s['score']
       
       total_loss_iterations += loss
       writer.add_scalar('Loss', loss,iter)
       writer.add_scalar('score', s['score'],iter) 

       if iter % 5000 == 0:
           avarage_loss= total_loss_iterations / 5000
           avg_score = final_score /5000
           total_loss_iterations = 0
           tqdm.write(f'iter :{iter} , Loss : {avarage_loss} , score :{avg_score}')
          #  scheduler.step()
           torch.save(model.state_dict(), f'/content/drive/MyDrive/HW4_Q1/{type}_Model.pt')
          
   return model

<ipython-input-22-4d2c4558e126>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric('sacrebleu')


# Test

In [23]:
def evaluate(model, input_lang, output_lang, sentences, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentences[0])
        output_tensor = tensorFromSentence(output_lang, sentences[1])

        decoded_words = []

        output = model(input_tensor, output_tensor)
        # print(output_tensor)

        for ot in range(output.size(0)):
            topv, topi = output[ot].topk(1)
            # print(topi)

            if topi[0].item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi[0].item()])
    return decoded_words

def evaluateRandomly(model, source, target, pairs, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('source {}'.format(pair[0]))
        print('target {}'.format(pair[1]))
        output_words = evaluate(model, source, target, pair)
        output_sentence = ' '.join(output_words)
        print('predicted {}'.format(output_sentence))

# train GRU

In [24]:
lang1 = 'M1'
lang2 = 'M2'
writer = SummaryWriter(log_dir="/content/drive/MyDrive/HW4_Q1/GRU_logs")
source, target, pairs = process_data(lang1, lang2)

randomize = random.choice(pairs)
print('random sentence {}'.format(randomize))

#print number of words
input_size = source.n_words
output_size = target.n_words
print('Input : {} Output : {}'.format(input_size, output_size))

embed_size = 256
hidden_size = 512
num_layers = 1
num_iteration = 100000

#create encoder-decoder model
encoder = Encoder(input_size, hidden_size, embed_size, num_layers,GRU=True)
decoder = Decoder(output_size, hidden_size, embed_size, num_layers,GRU=True)

model = Seq2Seq(encoder, decoder, device,mode='GRU').to(device)

#print model 
print(encoder)
print(decoder)

model = trainModel(model, source, target, pairs, num_iteration,type='GRU')
evaluateRandomly(model, source, target, pairs)

Read 49609 sentence pairs
Maximum lenght of input is 11
random sentence ['چو بخشنده ام بیش بسپارمش', 'کلاه از بر چرخ بگذارمش']
Input : 12687 Output : 13323
Encoder(
  (embedding): Embedding(12687, 256, padding_idx=2)
  (recurrent): GRU(256, 512)
)
Decoder(
  (embedding): Embedding(13323, 256, padding_idx=2)
  (recurrent): GRU(256, 512)
  (out): Linear(in_features=512, out_features=13323, bias=True)
  (softmax): LogSoftmax(dim=1)
)


  5%|▍         | 4999/100000 [02:54<54:59, 28.79it/s]

iter :5000 , Loss : 3.6835600317001482 , score :0.30085014668087895


 10%|█         | 10000/100000 [05:48<1:20:48, 18.56it/s]

iter :10000 , Loss : 3.498808099969217 , score :0.5929966326562244


...

iter :90000 , Loss : 3.3982938988049916 , score :5.151107958591581


 95%|█████████▍| 94998/100000 [57:47<03:01, 27.49it/s]

iter :95000 , Loss : 3.3996316342989608 , score :5.435722628486341


100%|██████████| 100000/100000 [1:00:52<00:00, 27.38it/s]


iter :100000 , Loss : 3.3889591455459644 , score :5.720267126350617
source بزد یک دم آن اژدهای پلید
target تنی چند ازیشان به دم درکشید
predicted که از و و PAD PAD PAD PAD PAD PAD PAD <EOS>
source ازان پس چو فرمایدم شهریار
target بیایم پرستش کنم بنده وار
predicted که از و و PAD PAD PAD PAD PAD PAD PAD <EOS>
source همی بوی مهر آمد از روی او
target همی زیب تاج آمد از موی او
predicted که از و و PAD PAD PAD PAD PAD PAD PAD <EOS>
source چنین گفت کز بارگاه بلند
target برفتم سوی رستم دیوبند
predicted که از و و PAD PAD PAD PAD PAD PAD PAD <EOS>
source بخوردند با شتاب چیزی که بود
target پس آنگه به زمزم بگفتند زود
predicted که از و و PAD PAD PAD PAD PAD PAD PAD <EOS>
source چو بشنید افراسیاب این سخن
target پشیمان شد از کرده های کهن
predicted که از و و PAD PAD PAD PAD PAD PAD PAD <EOS>
source چنین داد پاسخ که دانش بهست
target چو دانا بود برمهان برمهست
predicted که از و و PAD PAD PAD PAD PAD PAD PAD <EOS>
source گر ایدون که بر من نسازید بد
target کنید آنک از داد و گردی سزد
predicted که از و و PAD P

In [50]:
%reload_ext tensorboard

In [ ]:
# set correct path of GRU_logs in your system
%tensorboard --logdir "/content/drive/MyDrive/HW4_Q1/GRU_logs"

In [ ]:
!kill 64860

# train LSTM

In [25]:
lang1 = 'M1'
lang2 = 'M2'
writer = SummaryWriter(log_dir="/content/drive/MyDrive/HW4_Q1/LSTM_logs")
source, target, pairs = process_data(lang1, lang2)

randomize = random.choice(pairs)
print('random sentence {}'.format(randomize))

#print number of words
input_size = source.n_words
output_size = target.n_words
print('Input : {} Output : {}'.format(input_size, output_size))

embed_size = 256
hidden_size = 512
num_layers = 1
num_iteration = 100000

#create encoder-decoder model
encoder = Encoder(input_size, hidden_size, embed_size, num_layers,GRU=False)
decoder = Decoder(output_size, hidden_size, embed_size, num_layers,GRU=False)

model = Seq2Seq(encoder, decoder, device,mode='LSTM').to(device)

#print model 
print(encoder)
print(decoder)

model = trainModel(model, source, target, pairs, num_iteration,type='LSTM')
evaluateRandomly(model, source, target, pairs)

Read 49609 sentence pairs
Maximum lenght of input is 11
random sentence ['بدین کار در پارس گرد آمدند', 'بسی زین نشان داستانها زدند']
Input : 12687 Output : 13323
Encoder(
  (embedding): Embedding(12687, 256, padding_idx=2)
  (recurrent): LSTM(256, 512)
)
Decoder(
  (embedding): Embedding(13323, 256, padding_idx=2)
  (recurrent): LSTM(256, 512)
  (out): Linear(in_features=512, out_features=13323, bias=True)
  (softmax): LogSoftmax(dim=1)
)


  5%|▍         | 4998/100000 [03:11<1:00:26, 26.19it/s]

iter :5000 , Loss : 3.5659987035433542 , score :0.2978983691102564


 10%|▉         | 9999/100000 [06:26<58:31, 25.63it/s]

iter :10000 , Loss : 3.4073303403854442 , score :0.5829569743414826


...

iter :90000 , Loss : 3.3865597101847356 , score :5.142513847979084


 95%|█████████▍| 94998/100000 [1:00:45<03:03, 27.29it/s]

iter :95000 , Loss : 3.387362746270491 , score :5.427571669501144


100%|██████████| 100000/100000 [1:03:48<00:00, 26.12it/s]


iter :100000 , Loss : 3.373803162066143 , score :5.712629491023203
source گر آزار بودیش در دل ز من
target سرم برنیفراختی ز انجمن
predicted که و و و PAD PAD PAD PAD PAD PAD PAD <EOS>
source ز خویشان گزین کرد پیران هزار
target پذیره شدن را برآراست کار
predicted که و و و PAD PAD PAD PAD PAD PAD PAD <EOS>
source بیایید یکسر به درگاه من
target که بر مرز بگذشت بد خواه من
predicted که و و و PAD PAD PAD PAD PAD PAD PAD <EOS>
source تو بندیش هشیار و بگشای گوش
target سخن از خردمند مردم نیوش
predicted که و و و PAD PAD PAD PAD PAD PAD PAD <EOS>
source به تنها تن خویش جویم نبرد
target ز لشکر نخواهم کسی رنجه کرد
predicted که و و و PAD PAD PAD PAD PAD PAD PAD <EOS>
source نخست آفرین کرد بر کردگار
target جهاندار پیروز و پروردگار
predicted که و و و PAD PAD PAD PAD PAD PAD PAD <EOS>
source چنین داد پاسخ که از بیش خورد
target مگر آرزو بازگردد بدرد
predicted که و و و PAD PAD PAD PAD PAD PAD PAD <EOS>
source بدو گفت زال ای پسر این سخن
target مگوی و جدا کن سرش را ز بن
predicted که و و و PAD PAD PAD PAD PAD 

In [ ]:
# set correct path of GRU_logs in your system
%tensorboard --logdir "/content/drive/MyDrive/HW4_Q1/LSTM_logs"